In [8]:
from pathlib import Path
import librosa
import torch
import pandas as pd

In [2]:
from sonics import HFAudioClassifier
from sonics.utils.dataset import AudioDataset
model = HFAudioClassifier.from_pretrained("awsaf49/sonics-spectttra-gamma-5s")

/home/tomek/github/wimu-sonics/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
from gradio_client import Client, handle_file
LOCAL = False


def get_predictions_hf(audio_files):
    results = []
    client = Client("awsaf49/sonics-fake-song-detection")
    for file in audio_files:
        result = client.predict(
                audio_file=handle_file(file),
                model_type="SpecTTTra-γ",
                duration="5s",
                api_name="/predict"
        )
        results.append(result["label"])
    return results

def get_predictions(audio_files):
    predictions = []
    for audio in audio_files:
        predictions.append(model(audio["audio"][None,:],).item())
    return predictions

results = {}
with torch.no_grad():
    for folder in Path("../data/examples").iterdir():
        print("running",folder.name)
        all_audio = list(folder.glob("*"))[:10]
        if LOCAL:
            dataset = AudioDataset(all_audio, labels=[0]*10, random_sampling=False, max_len=32000*10)
            result = get_predictions(dataset)
        else:
            result = get_predictions_hf(all_audio)
        results[folder.name] = result

df = pd.DataFrame(results)
df

running musicgen
Loaded as API: https://awsaf49-sonics-fake-song-detection.hf.space ✔
running suno
Loaded as API: https://awsaf49-sonics-fake-song-detection.hf.space ✔
running YuE
Loaded as API: https://awsaf49-sonics-fake-song-detection.hf.space ✔
running real
Loaded as API: https://awsaf49-sonics-fake-song-detection.hf.space ✔
running udio
Loaded as API: https://awsaf49-sonics-fake-song-detection.hf.space ✔


,musicgen,suno,YuE,real,udio
0,Real,Real,Real,Real,Real
1,Real,Fake,Real,Real,Real
2,Real,Fake,Real,Real,Real
3,Real,Fake,Real,Real,Fake
4,Real,Fake,Real,Real,Real
5,Real,Fake,Fake,Real,Real
6,Real,Real,Real,Real,Real
7,Real,Fake,Real,Real,Real
8,Real,Fake,Real,Real,Real
9,Real,Fake,Fake,Real,Real
